In [12]:
import os
from IPython.core.display import display, HTML, Markdown, clear_output, Javascript
import ipywidgets as widgets
from IPython.display import IFrame, display

def on_change(change): # si attiva quando cambio scelta dell'elemento da selezionare
    global attach
    if change['type'] == 'change' and change['name'] == 'value':
        #print ("Stai scegliendo %s" % change['new'])
        attach = change['new']
        return attach

def on_button_update(_):
    clear_output()
    #display(Javascript('''var c = IPython.notebook.get_selected_index();
                       #IPython.notebook.execute_cells([c])'''))
    allegato()

def open_attachment(attach_name):
    path_attachments = "allegati"
    path_file=str(path_attachments + "/" + attach_name)
    ext = os.path.splitext(path_file)[-1].lower()
    #print(path_file)
    #print(ext)
    #print(ext.replace('.', ''))
    if(ext=='.pdf'):
        pdf=IFrame(path_file, width=700, height=400)
        display(pdf)
    else:
        #file = open(path_file, "rb")
        display(Markdown("![alt text](" + path_file + ")"))
        delete = widgets.Button(description='Rimuovi allegato')
        delete.on_click(on_button_delete)
        display(delete)

        #image = file.read()
        #w=widgets.Image(
        #    value=image,
        #    format=ext.replace('.', ''),
        #    width=300,
        #    height=400,
        #)
        #display(w)

def on_button_confirm(_):
    global attach
    clear_output()
    print('Allegato: ' + str(attach))
    open_attachment(attach)

def allegato():
    path_attachments = "allegati"
    global attach # allegato attuale scelto
    update = widgets.Button(description='Aggiorna') #bottone visualizzato in ogni caso
    delete = widgets.Button(description='Rimuovi allegato')
    attach_list = os.listdir(path_attachments)
    if 'ck_points' in attach_list:
        attach_list.remove('ck_points')

    if(attach_list): #si attiva se c'è almeno un elemento in lista quindi nella cartella allegati
        confirm = widgets.Button(description='Conferma')
        chosen_attach_wid = widgets.Dropdown(
            options = attach_list,
            value = attach_list[0],
            description = 'Allegati:',
            disabled = False,
        )
        attach = attach_list[0]
        chosen_attach_wid.observe(on_change)
        update.on_click(on_button_update)
        confirm.on_click(on_button_confirm)
        delete.on_click(on_button_delete)
        all_buttons = [update, chosen_attach_wid, confirm, delete]
        display(widgets.HBox(all_buttons))
    else:
        print(f"ATTENZIONE: non sono presenti file nella cartella allegati.") #Ti ricordiamo che la cartella utile per gli allegati da te prodotti per questo esercizio in modo_libero di svolgimento è la cartella @path_ex_folder@/modo_libero/allegati/")
        update.on_click(on_button_update)
        delete.on_click(on_button_delete)
        all_buttons = [update, delete]
        display(widgets.HBox(all_buttons))

def on_button_delete(_):
    #delete_above_cell()
    delete_this_cell()

def delete_above_cell():
    display(Javascript('''var c = IPython.notebook.get_selected_index();
                                IPython.notebook.get_cell(c-1).metadata.editable = true;
                                IPython.notebook.get_cell(c-1).metadata.deletable = true;
                                IPython.notebook.delete_cell([c-1]);'''))
def delete_this_cell():
    display(Javascript('''var c = IPython.notebook.get_selected_index();
                                IPython.notebook.get_cell(c).metadata.editable = true;
                                IPython.notebook.get_cell(c).metadata.deletable = true;
                                IPython.notebook.delete_cell([c]);'''))

def on_button_raw_attach(_):
    add_attach_cell_din('allegato()')

def on_button_md(_):
    add_md_cell_din()

def on_button_code(_):
    add_code_cell_din()

def on_button_raw(_):
    add_raw_cell_din()

def add_raw_cell_din():
    display_id = int(time.time()*1e9) # Hack
    display(Javascript('''var idx = IPython.notebook.get_selected_index();
                       var c = IPython.notebook.insert_cell_at_index("raw", idx);
    c.set_text('');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_raw(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.get_cell(t_index).metadata.deletable = true;
    IPython.notebook.get_cell(t_index).set_text('');;'''),display_id=display_id) # Hack
    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)
    return

def add_attach_cell_din(code = ''):
    display_id = int(time.time()*1e9) # Hack
    display(Javascript('''var c = IPython.notebook.insert_cell_above();
    c.set_text(' ''' + code + ''' ');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_code(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.execute_cells([t_index]);
    IPython.notebook.get_cell(t_index).metadata.editable = false;'''),display_id=display_id) # Hack
    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)
    return

def add_code_cell_din(code = ''):
    display_id = int(time.time()*1e9)
    display(Javascript('''var c = IPython.notebook.insert_cell_above();
    c.set_text(' ''' + code + ''' ');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_code(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.execute_cells([t_index]);
    IPython.notebook.get_cell(t_index).metadata.deletable = true;
    IPython.notebook.get_cell(t_index).set_text('');'''),display_id=display_id)
    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)
    return

def add_md_cell_din():
    display_id = int(time.time()*1e9)
    display(Javascript('''var c = IPython.notebook.insert_cell_above();
    c.set_text(' ');
    var t_index = IPython.notebook.get_cells().indexOf(c);
    IPython.notebook.to_markdown(t_index);
    IPython.notebook.get_cell(t_index).render();
    IPython.notebook.get_cell(t_index).metadata.deletable = true;
    IPython.notebook.get_cell(t_index).set_text('');'''),display_id=display_id) # Hack
    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)
    return

def add_cell(code='', position='below', celltype='markdown', is_execute = False):
    """Create a cell in the IPython Notebook.
    code: unicode, Code to fill the new cell with.
    celltype: unicode, Type of cells "code" or "markdown".
    position: unicode, Where to put the cell "below" or "at_bottom"
    is_execute: boolean, To decide if the cell is executed after creation
    """

    # Create a unique id based on epoch time
    display_id = int(time.time()*1e9)

    if is_execute:
        display(Javascript("""
        var basis = IPython.notebook.insert_cell_{0}("{1}");
        basis.set_text(atob("{2}"));
        basis.execute();
        """.format(position, celltype, " ")),display_id=display_id)

    else:
        display(Javascript("""
        var basis = IPython.notebook.insert_cell_{0}("{1}");
        basis.set_text(atob("{2}"));
        """.format(position, celltype, " ")),display_id=display_id)


    # Necessary hack to avoid self-generation of cells at notebook re-opening
    # See http://tiny.cc/fnf3nz
    display(Javascript(""" """), display_id=display_id, update=True)

def code_button_delete():
    button_delete = widgets.Button(description="Rimuovi l'allegato", tooltip="Seleziona la cella e clicca su Elimina")
    button_delete.on_click(on_button_delete)
    #print("Vuoi eliminare la cella selezionata?")
    display(button_delete)

def loader_main():
    button_raw_attach = widgets.Button(description="Rispondi (Allegato)", tooltip="Collega un file dalla cartella attachments")
    button_md = widgets.Button(description="Rispondi (Markdown)", abstooltip="Aggiungi una cella per scrivere del testo in Markdown")
    button_code = widgets.Button(description="Rispondi (Code)", tooltip="Aggiungi una cella per scrivere del codice in Python")
    button_raw = widgets.Button(description="Rispondi (Raw)", tooltip="Aggiungi una cella per scrivere del testo libero (SCONSIGLIATA: righe troppo lunghe potrebbero fuoriuscire nella rendition, usala solo se non riesci a controllare il Markdown)")
    button_raw_attach.on_click(on_button_raw_attach)
    button_md.on_click(on_button_md)
    button_code.on_click(on_button_code)
    button_raw.on_click(on_button_raw)

    all_buttons = [button_code, button_md, button_raw, button_raw_attach]
    display(widgets.HBox(all_buttons))


# ESERCIZIO: Conteggio di cammini in una griglia rettangolare con celle proibite #


In [13]:
import os
from IPython.core.display import display, HTML, Markdown, clear_output, Javascript
import ipywidgets as widgets
from IPython.display import IFrame, display
import numpy as np
import copy

In [14]:
    from tabulate import tabulate
    import copy
    n_tasks = 6;
    arr_point = [-1] * n_tasks;

In [15]:

    campo_minato = [[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '*'], [' ', ' ', ' ', '*', ' ', '*', ' ', ' ', ' '], [' ', ' ', '*', '*', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', '*', ' '], [' ', ' ', '*', ' ', '*', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', '*', ' ', ' '], [' ', '*', ' ', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']]
    m = len(campo_minato)
    n = len(campo_minato[0])
    mappa = [ ["*"]*(n+1) ] + [ (["*"] + r) for r in campo_minato]
    

In [16]:
    def visualizza(env):
        if len(env)==m+1 and len(env[0])==n+1:
            index=[chr(65+i) for i in range(m)]
            aux=[r[1:] for r in env[1:]]


        if len(env)==m+2 and len(env[0])==n+2:
            index=[chr(65+i) for i in range(m)]
            aux=[r[1:-1] for r in env[1:-1]]

        columns=[str(i) for i in range(1,n+1)]
        print(tabulate(aux, headers=columns, tablefmt='fancy_grid', showindex=index))


    def evaluation_format(answ, pt_green,pt_red, index_pt):
        pt_blue=0
        if pt_green!=0:
            if answ == "Si":
                pt_green=pt_red
                pt_red=0
                pt_blue=pt_green
            else:
                pt_blue=pt_red-pt_green
                pt_red=0
        arr_point[index_pt]=pt_green
        file = open("points.txt", "w")
        file.write(str(arr_point))
        file.close()
        return f"{answ}. Totalizzeresti <span style='color:green'>[{pt_green} safe pt]</span>,                                         <span style='color:blue'>[{pt_blue} possible pt]</span>,                                         <span style='color:red'>[{pt_red} out of reach pt]</span>.<br>"

    def check_num_paths_to(mappa, num_paths_to, pt_green, pt_red, index_pt, return_only_boolean=False):
        if len(num_paths_to) != m+1:
            if return_only_boolean:
                    return False
            return evaluation_format("No", 0, pt_red,index_pt)+f"Le righe della matrice $num\_paths\_to$ devono essere $m+1=${m+1}, non {len(num_paths_to)}."
        if len(num_paths_to[0]) != n+1:
            if return_only_boolean:
                    return False
            return evaluation_format("No", 0, pt_red, index_pt)+f"Le colonne della matrice $num\_paths\_to$ devono essere $n+1=${n+1}, non {len(num_paths_to[0])}."

        for i in range (0,m):
            if num_paths_to[i][0]!=0:
                if return_only_boolean:
                    return False
                return evaluation_format("No", 0, pt_red, index_pt)+f"Attenzione, i cammini devono partire dalla cella $(1,1)$ e pertanto $num\_paths\_to[${i}$][0] = 0$"
        for j in range (0,n):
            if num_paths_to[0][j]!=0:
                if return_only_boolean:
                    return False
                return evaluation_format("No", 0, pt_red, index_pt)+f"Attenzione, i cammini devono partire dalla cella $(1,1)$ e pertanto $num\_paths\_to[0][${j}$] = 0$"
        num_paths_to_forgiving = copy.deepcopy(num_paths_to)
        num_paths_to_forgiving[1][1] = 1
        for i in range(m,0,-1):
            for j in range (n,0,-1):
                if i==1 and j==1:
                    if return_only_boolean:
                        return True
                    return  evaluation_format("Si", pt_green, pt_red, index_pt)+"Non riscontro particolari problemi della tua versione della matrice $num\_paths\_to$."
                if mappa[i][j]!="*":
                    if num_paths_to_forgiving[i][j]!=num_paths_to_forgiving[i-1][j]+num_paths_to_forgiving[i][j-1]:
                        if return_only_boolean:
                            return False
                        return  evaluation_format("No", 0, pt_red, index_pt)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$."
                elif num_paths_to_forgiving[i][j]!=0:
                    if return_only_boolean:
                        return False
                    return  evaluation_format("No", 0, pt_red, index_pt)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$."


    def check_num_paths_from(mappa, num_paths_from, pt_green, pt_red, index_pt, return_only_boolean=False):
        if len(num_paths_from) != m+2:
            if return_only_boolean:
                return False
            return evaluation_format("No", 0, pt_red, index_pt)+f"Le righe della matrice $num\_paths\_from$ devono essere $m+2=${m+2}, non {len(num_paths_from)}."
        if len(num_paths_from[0]) != n+2:
            if return_only_boolean:
                    return False
            return evaluation_format("No", 0, pt_red, index_pt)+f"Le colonne della matrice $num\_paths\_from$ devono essere $n+2=${n+2}, non {len(num_paths_from[0])}."

        for i in range (0,m+1):
            if num_paths_from[i][n+1]!=0:
                if return_only_boolean:
                    return False
                return evaluation_format("No", 0, pt_red, index_pt)+f"Attenzione, i cammini devono partire dalla cella $(8,9)$ e pertanto $num\_paths\_from[${i}$][10] = 0$"
        for j in range (0,n+1):
            if num_paths_from[m+1][j]!=0:
                if return_only_boolean:
                    return False
                return evaluation_format("No", 0, pt_red, index_pt)+f"Attenzione, i cammini devono partire dalla cella $(8,9)$ e pertanto $num\_paths\_from[9][${j}$] = 0$"
        num_paths_from_forgiving = copy.deepcopy(num_paths_from)
        num_paths_from_forgiving[m][n] = 1
        for i in range(1,m-1):
            for j in range (1,n-1):
                if mappa[i][j]!="*":
                    if num_paths_from_forgiving[i][j]!=num_paths_from_forgiving[i+1][j]+num_paths_from_forgiving[i][j+1]:
                        if return_only_boolean:
                            return False
                        return  evaluation_format("No", 0, pt_red, index_pt)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_from$."
                elif num_paths_from_forgiving[i][j]!=0:
                    if return_only_boolean:
                        return False
                    return  evaluation_format("No", 0, pt_red, index_pt)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_from$."
        for i in range (1, m):
            if mappa[i][n]!="*":
                if num_paths_from_forgiving[i][n]!=num_paths_from_forgiving[i+1][n]+num_paths_from_forgiving[i][n+1]:
                    if return_only_boolean:
                        return False
                    return  evaluation_format("No", 0, pt_red, index_pt)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_from$."
            elif num_paths_from_forgiving[i][n]!=0:
                if return_only_boolean:
                    return False
                return  evaluation_format("No", 0, pt_red, index_pt)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_from$."
        for j in range (1, n):
            if mappa[m][j]!="*":
                if num_paths_from_forgiving[m][j]!=num_paths_from_forgiving[m+1][j]+num_paths_from_forgiving[m][j+1]:
                    if return_only_boolean:
                        return False
                    return  evaluation_format("No", 0, pt_red, index_pt)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_from$."
            elif num_paths_from_forgiving[m][j]!=0:
                if return_only_boolean:
                    return False
                return  evaluation_format("No", 0, pt_red, index_pt)+"Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_from$."
        if return_only_boolean:
            return True
        return  evaluation_format("Si", pt_green, pt_red, index_pt)+"Non riscontro particolari problemi della tua versione della matrice $num\_paths\_from$."

    def Latex_type(string):
        return string.replace("_", "\_")

    def visualizza_e_valuta(nome_matrice, matrice, pt_green, pt_red, index_pt):
        display(Markdown(f"La tua versione attuale della matrice ${Latex_type(nome_matrice)}$ è la seguente:"))
        visualizza(matrice)
        display(Markdown(f"<b>Validazione della tua matrice ${Latex_type(nome_matrice)}$:</b>"))
        display(Markdown(eval(f"check_{nome_matrice}(mappa,matrice,pt_green, pt_red, index_pt)")))
    

<b>Nota</b>: Saper programmare non è la competenza che intendiamo valutare con questo esercizio.
    Decidi tu, in piena libertà, se preferisci compilare le tabelle e le risposte a mano, oppure scrivere del codice che lo faccia per te
    o che ti assista nelle misura che ti è più utile. Sei incoraggiato a ricercare l'approccio per te più pratico, sicuro e conveniente.
    Non verranno pertanto attribuiti punti extra per chi scrive del codice. I punti ottenuti dalle risposte consegnate a chiusura sono l'unico elemento oggetto di valutazione.
    In ogni caso, il feedback offerto dalle procedure di validazione rese disponibili è di grande aiuto.
    Esso convalida la conformità delle tue risposte facendo anche presente a quanti dei punti previsti  le tue risposte possono ambire.
    Per facilitare chi di voi volesse scrivere del codice a proprio supporto, abbiamo aggiunto alla mappa di $m$ righe ed $n$ colonne una riga e colonna iniziale (di indice zero), fatte interamente di mine, perchè non si crei confusione col fatto che gli indici di liste e array in programmazione partono da zero.
    

Un robot, inizialmente situato nella cella $A1=(1, 1)$, deve portarsi nella cella $H9=(8,9)$.Le celle che riportano il simbolo '*' contengono una mina od altre trapole mortali, ed il robot deve evitarle.I movimenti base possibili sono il passo verso destra (ad esempio il primo passo potrebbe avvenire dalla cella $A1$ alla cella $A2$) ed il passo verso il basso (ad esempio, come unica altra alternativa per il primo passo il robot potrebbe portarsi quindi nella cella $B1$).Quanti sono i possibili percorsi che può fare il robot per andare dalla cella $A1=(1, 1)$ alla cella $H9=(8,9)$?

In [17]:

    from tabulate import tabulate
    print(tabulate([[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '*'], [' ', ' ', ' ', '*', ' ', '*', ' ', ' ', ' '], [' ', ' ', '*', '*', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', '*', ' '], [' ', ' ', '*', ' ', '*', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', '*', ' ', ' '], [' ', '*', ' ', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']], headers=['1', '2', '3', '4', '5', '6', '7', '8', '9'], tablefmt="fancy_grid", showindex=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']))

╒════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╕
│    │ 1   │ 2   │ 3   │ 4   │ 5   │ 6   │ 7   │ 8   │ 9   │
╞════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╡
│ A  │     │     │     │     │     │     │     │     │ *   │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ B  │     │     │     │ *   │     │ *   │     │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ C  │     │     │ *   │ *   │     │     │     │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ D  │     │     │     │     │     │     │     │ *   │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ E  │     │     │ *   │     │ *   │     │     │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ F  │     │     │     │     │     │     │ *   │     │     │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ G  │     │ *   │     │     │     │     │     │     │     │
├────┼─────┼─────┼─────┼

__Richiesta 1 [5 punti]__: A mano o tramite un programma componi la matrice $num\_paths\_to$ di dimensione $(m+1)\times(n+1)$ e tale per cui in $num\_paths\_to[i][j]$ sia riposto il numero di cammini dalla cella $A1=(1,1)$ alla generica cella $(i,j)$, per ogni $i = 0,..., m+1$ e $j = 0,..., n+1$.

In [7]:
num_paths_to=[
		[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
		[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],		# [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '*']
		[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],		# [' ', ' ', ' ', '*', ' ', '*', ' ', ' ', ' ']
		[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],		# [' ', ' ', '*', '*', ' ', ' ', ' ', ' ', ' ']
		[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],		# [' ', ' ', ' ', ' ', ' ', ' ', ' ', '*', ' ']
		[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],		# [' ', ' ', '*', ' ', '*', ' ', ' ', ' ', ' ']
		[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],		# [' ', ' ', ' ', ' ', ' ', ' ', '*', ' ', ' ']
		[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],		# [' ', '*', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
		[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],		# [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
]

In [8]:
visualizza_e_valuta('num_paths_to',num_paths_to, pt_green=5, pt_red=5,index_pt=0)

La tua versione attuale della matrice $num\_paths\_to$ è la seguente:

╒════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╕
│    │   1 │   2 │   3 │   4 │   5 │   6 │   7 │   8 │   9 │
╞════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╡
│ A  │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ B  │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ C  │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ D  │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ E  │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ F  │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ G  │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │
├────┼─────┼─────┼─────┼

<b>Validazione della tua matrice $num\_paths\_to$:</b>

No. Totalizzeresti <span style='color:green'>[0 safe pt]</span>,                                         <span style='color:blue'>[0 possible pt]</span>,                                         <span style='color:red'>[5 out of reach pt]</span>.<br>Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_to$.

__Richiesta 2 [5 punti]__: Componi ora una matrice $num\_paths\_from$ di dimensione $(m+2)\times(n+2)$ e tale per cui in $num\_paths\_from[i][j]$, per ogni $i = 1,..., m+1$ e $j = 1,..., n+1$, sia riposto il numero di cammini dalla generica cella $(i,j)$ alla cella $H9=(8,9)$.

In [9]:
num_paths_from=[
		[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
		[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],		# [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '*']
		[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],		# [' ', ' ', ' ', '*', ' ', '*', ' ', ' ', ' ']
		[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],		# [' ', ' ', '*', '*', ' ', ' ', ' ', ' ', ' ']
		[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],		# [' ', ' ', ' ', ' ', ' ', ' ', ' ', '*', ' ']
		[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],		# [' ', ' ', '*', ' ', '*', ' ', ' ', ' ', ' ']
		[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],		# [' ', ' ', ' ', ' ', ' ', ' ', '*', ' ', ' ']
		[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],		# [' ', '*', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
		[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],		# [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
		[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
]

In [10]:
visualizza_e_valuta('num_paths_from',num_paths_from, pt_green=5, pt_red=5,index_pt=1)

La tua versione attuale della matrice $num\_paths\_from$ è la seguente:

╒════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╤═════╕
│    │   1 │   2 │   3 │   4 │   5 │   6 │   7 │   8 │   9 │
╞════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╡
│ A  │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ B  │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ C  │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ D  │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ E  │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ F  │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ G  │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │   0 │
├────┼─────┼─────┼─────┼

<b>Validazione della tua matrice $num\_paths\_from$:</b>

No. Totalizzeresti <span style='color:green'>[0 safe pt]</span>,                                         <span style='color:blue'>[0 possible pt]</span>,                                         <span style='color:red'>[5 out of reach pt]</span>.<br>Ti avviso: riscontro dei problemi nella tua versione della matrice $num\_paths\_from$.

__Richiesta 3 [5 punti]__: Quanti sono i percorsi con partenza in $A1=(1,1)$ e arrivo in $H9=(8,9)$.

Inserisci la risposta

__Richiesta 4 [10 punti]__: Quanti sono i percorsi con partenza in $B5=(2, 5)$ e arrivo in $H9=(8,9)$.

Inserisci la risposta

__Richiesta 5 [10 punti]__: Quanti sono i percorsi con partenza in $A1=(1,1)$ e arrivo in $F6=(6, 6)$?

Inserisci la risposta

__Richiesta 6 [10 punti]__: Quanti sono i percorsi che partono da $A1=(1,1)$, passano da $D5=(4, 5)$, e arrivano in $H9=(8,9)$?

Inserisci la risposta

In [11]:
import os
import time
from urllib.request import urlopen
import nbformat
from traitlets.config import Config
from nbconvert import HTMLExporter
from nbconvert import RSTExporter, NotebookExporter
from IPython.display import Image, HTML, Javascript
from datetime import datetime

import shutil
last ='preview_last/' #path of last submission
old = 'preview_old/' #path of prevous submissions

#hide alert buttons
def hide_w(w):
    for e in w:
        e.layout.visibility = 'hidden'

#show alert buttons
def show_w(w):
    for e in w:
        e.layout.visibility = 'visible'

def are_you_sure():
    show_w(alert_buttons)

label = widgets.Label(value="Un tuo elaborato era già stato precedentemente prodotto. Vuoi rimpiazzarlo con il presente?")
yes = widgets.Button(description="Si", tooltip="Produce il nuovo elaborato. Esso diviene la versione attuale prendendo il posto del precedente")
no = widgets.Button(description="No")

alert_buttons = [label,yes,no]
hide_w(alert_buttons)

#move all files from last to old and save the new preview
def yes_clicked(b):
    files = os.listdir(last)
    for f in files:
        if f == '.ipynb_checkpoints':
            continue
        if not os.path.isdir('./preview_old'):
            os.mkdir(old)
        shutil.move(last+f, old)
    display(Javascript('IPython.notebook.save_checkpoint();'))
    now = datetime.now() # current date and time
    date_time = now.strftime("%Y-%m-%d-%H-%M-%S")
    hname = 'Esercizio_04.ipynb'[:-6] + '_' + date_time
    ! jupyter nbconvert Esercizio_04.ipynb --to html_embed --output=$hname --output-dir=./preview_last/ --TagRemovePreprocessor.enabled=True --TagRemovePreprocessor.remove_cell_tags="['noexport']"
    display(Javascript('window.open("./preview_last/' + hname + '.html")'))
    hide_w(alert_buttons)
def no_clicked(b):
    hide_w(alert_buttons)


def generate_preview_HTML(_):
    if not os.path.isdir('./preview_last'):
        os.mkdir(last)
    directory= os.listdir(last)
    if len(directory) <1:
        display(Javascript('IPython.notebook.save_checkpoint();'))
        now = datetime.now() # current date and time
        date_time = now.strftime("%Y-%m-%d-%H-%M-%S")
        hname = 'Esercizio_04.ipynb'[:-6] + '_' + date_time
        ! jupyter nbconvert Esercizio_04.ipynb --TagRemovePreprocessor.enabled=True --TagRemovePreprocessor.remove_cell_tags="['noexport']" --to html_embed --output=$hname --output-dir=./preview_last/
        display(Javascript('window.open("./preview_last/' + hname + '.html")'))
    else:
        are_you_sure()


button = widgets.Button(description="Salva & Esporta", tooltip="Esporta il foglio Jupyter in HTML nella cartella preview")
output = widgets.Output()
button.on_click(generate_preview_HTML)
yes.on_click(yes_clicked)
no.on_click(no_clicked)
h_box1 = widgets.HBox([button])
h_box2 = widgets.HBox([label,yes,no])
display(widgets.VBox([h_box1,h_box2]))
